# Parabolic Focus Sequence

performs fine sampling of z-axis hexapod translations. 

## Set up

### Import libraries

In [ ]:
import sys
import asyncio
import time
import os

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy
from datetime import date

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.observatory.control.utils import RotType

from lsst.ts.idl.enums.Script import ScriptState

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

### Setting up logger

In [ ]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

### Getting unique index for script

In [ ]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

### Instanciate Script

In [ ]:
script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_task

### Forward ATCS and LATISS

In [ ]:
atcs = script.atcs
latiss = script.latiss

### Set up script log level

In [ ]:
script.log.level = logging.DEBUG

---
## Parabolic Focus Data Acquisition

### Configuration

#### ATSpectrograph configuration

Get available instrument configurations:

In [ ]:
inst_setup = await script.latiss.get_available_instrument_setup()
logger.info(f'filters are: {inst_setup[0]},\ngratings are: {inst_setup[1]}')

Edit in the cell below the configuration of the instrument for the focus sequence. Both variables are strings. 

In [ ]:
Filter_to_focus = 'SDSSg'
Grating_to_focus = 'holo4_003'

#### ATAOS corrections

Check that corrections are enabled. 


In [ ]:
print(await script.atcs.rem.ataos.evt_correctionEnabled.aget())

If False, please enable corrections with the following command


In [ ]:
await script.atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

### Slew to target

Introduce target name

In [ ]:
target = 'HD 110304'

or query for a target at the chosen coordinates


In [ ]:
target = await script.atcs.find_target(az=120, el = 60, mag_limit=8)

Slew to the defined target

In [ ]:
await script.atcs.slew_object(target, rot_type=RotType.PhysicalSky)

Take a test image with the setup configuration. 

In [ ]:
await script.latiss.take_engtest(2, filter=Filter_to_focus, grating=Grating_to_focus)

### Acquiring Data

Fine-step focus sweep from -5x to 5x.

In [ ]:
original_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(f'Original focus offset is \n {original_focus_offset}')

In [ ]:
z_offset_start = -0.1 # mm
z_offset_step = 0.025 # mm
z_offset_end = -z_offset_start

steps = np.arange(z_offset_start, z_offset_end + 0.01, z_offset_step)
parabolic_focus_data = []

await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_start)
    
await asyncio.sleep(2)
for step in steps:
    print(f"Total z offset = {step:.3f}")
        
    exposure = await latiss.take_engtest(
        1, 1, filter=Filter_to_focus, grating=Grating_to_focus)
    parabolic_focus_data.append(exposure[0])
    
    print(f'Current focus offset \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')
    
    await atcs.rem.ataos.cmd_offset.set_start(z=z_offset_step)

# Put offset back to original position
await atcs.rem.ataos.cmd_offset.set_start(z= -(z_offset_end + z_offset_step))
print(f'Back to original position \n {await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()}')


In [ ]:
current_focus_offset = await script.atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(current_focus_offset)

#### Save results 

In [ ]:
filename = "FocusSequence"+date.today().strftime("%y%m%d")+'_'+ Filter_to_focus+'+'+Grating_to_focus+'.txt'
print(f'The sequence of images will be saved in {filename}')

In [ ]:
file = open(filename, "w+")
file.write(str(parabolic_focus_data))
file.close()

### Stop tracking

In [ ]:
await script.atcs.stop_tracking()

## CWFS Focusing 

### Set up configuration

In [ ]:
configuration = yaml.safe_dump({"filter": Filter_to_focus, 
                                "grating": Grating_to_focus,
                                "exposure_time": 20,})

script = LatissCWFSAlign(index=index, remotes=True)  # this essentially calls the init method
await script.start_taskSet script state to `UNCONFIGURED`.    
The next line is not required the first time the script is run, however, in each additional instance the cell is run, an error will be thrown if it is not included.  
Therefore, it is included here despite being a non-operation in the first instance.  

In [ ]:
script.set_state(ScriptState.UNCONFIGURED)

### Put the ScriptState to CONFIGURED

In [ ]:
config_data = script.cmd_configure.DataType()
config_data.config = configuration
await script.do_configure(config_data)

Set these script parameters to None to verify a re-reduction does not happen of the images.

In [ ]:
script.intra_visit_id = None
script.extra_visit_id = None
script.short_timeout = 10

### Set groupID and launch the script

This sets the same group ID for all exposures taken in the script.

In [ ]:
group_id_data = script.cmd_setGroupId.DataType(
    groupId=astropy.time.Time.now().isot)

await script.do_setGroupId(group_id_data)
await script.arun()

### Stop tracking

In [ ]:
await script.atcs.stop_tracking()